In [13]:
import os 
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key=os.environ["OPENAI_API_KEY"]

In [14]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)

In [15]:
import warnings
from langchain._api import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [16]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is blue.")
]

Call the ChatModel(LLM)

In [17]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is such a calming and serene color. It's often associated with peace, tranquility, and stability. Plus, there are so many shades of blue to choose from, ranging from light and airy to deep and rich. It's a versatile color that can be incorporated into many different design aesthetics. What do you like most about the color blue?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 13, 'total_tokens': 83, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6e35a561-9a41-4ce6-af98-f0987856e07e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 70, 'total_tokens': 83})

Check if the Chatbot remembers the favourite color

In [18]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm not sure, what is your favorite color?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a9690125-7fec-46d8-aaf0-1c0e9722877d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24})

Add memory to Chatbot

In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]

chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot,
    get_session_history
)

In [20]:
session1 = {"configurable": {"session_id": "001"}}

In [23]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favourite food is pizza.")],
    config=session1,
)

responseFromChatbot.content

'Great choice! Pizza is a popular favourite for many people due to its delicious and versatile nature. What do you enjoy most about pizza?'

In [24]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favourite food?")],
    config=session1,
)

responseFromChatbot.content

"Your favourite food is pizza! It's a popular choice for many people due to its delicious and versatile nature. What do you enjoy most about pizza?"

Session_id change 


In [25]:
session2 = {"configurable": {"session_id": "002"}}

In [26]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favourite food?")],
    config=session2,
)

responseFromChatbot.content

"I'm sorry, I am not able to determine your favourite food as I am an AI assistant and do not have personal information about you."

Go back to session1

In [27]:
session1 = {"configurable": {"session_id": "001"}}

In [28]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favourite food?")],
    config=session1,
)

responseFromChatbot.content

"Based on your previous responses, it seems like your favourite food is pizza. It's a popular choice for many people due to its delicious and versatile nature. What do you enjoy most about pizza?"

Chatbot has session memory now. let's check if it remembers conversation from second session

In [29]:
session2 = {"configurable": {"session_id": "002"}}

In [30]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My name is John.")],
    config=session2,
)

responseFromChatbot.content

'Nice to meet you, John! How can I assist you today?'

In [31]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=session2
)

responseFromChatbot.content

'Your name is John.'

In [32]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favourite food?")],
    config=session1,
)

responseFromChatbot.content

"Based on your previous responses, it seems like your favourite food is pizza. It's a popular choice for many people due to its delicious and versatile nature. What do you enjoy most about pizza?"

Chatbot remebers both conversations

In [33]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favourite food is pizza.'), AIMessage(content="I love the combination of melted cheese, tangy tomato sauce, and a variety of toppings like pepperoni, mushrooms, and bell peppers. Whether it's a classic Margherita or a loaded meat lovers, pizza is always a satisfying and comforting meal for me. Plus, it's easy to share with friends and family, making it a great option for gatherings or parties. Overall, pizza is my go-to comfort food and will always be a favourite of mine.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 13, 'total_tokens': 106, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-abb5202c-1e33-4384-a215-2c15fea4ea10-0', usage_metadata={'input_tokens': 13, 'output_tokens': 

In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt
    | chatbot
)

In [35]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages"
)

Let's add 2 more messages to session1 conversation

In [36]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite car is BMW X5")],
    config=session1,
)

responseFromChatbot.content

"That's great to hear! The BMW X5 is a popular luxury SUV known for its sleek design, high performance, and advanced technology features. It's a stylish and comfortable choice for those who appreciate quality and luxury in their vehicles. What do you love most about the BMW X5?"

In [37]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favourite sport is MMA.")],
    config=session1,
)

responseFromChatbot.content

"That's awesome! MMA (Mixed Martial Arts) is a dynamic and intense combat sport that combines various fighting techniques from disciplines such as boxing, Brazilian Jiu-Jitsu, Muay Thai, and wrestling. It requires strength, skill, and strategy, making it a thrilling and exciting sport to watch and practice. What do you enjoy most about MMA?"

The chatbot has now 4 messages. Let's check the Chatbot with limited memory

In [38]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="What is my favourite food?")],
    },
    config=session1,
)

responseFromChatbot.content

"I'm sorry, but as an assistant, I don't have access to personal information about you, including your favorite food. However, if you'd like to share your favorite food with me, I'd be happy to discuss it with you or provide information about it!"

Let's check the Chatbot with unlimited memory

In [40]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favourite food?")],
    config=session1,
)

responseFromChatbot.content

"Based on our conversation, it seems like your favorite food is pizza. It's a popular choice for many people due to its delicious and versatile nature. What do you enjoy most about pizza?"

In [41]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favourite sport?")],
    config=session1,
)

responseFromChatbot.content

'Based on our conversation, it seems like your favorite sport is MMA (Mixed Martial Arts). MMA is a dynamic and intense combat sport that combines various fighting techniques from disciplines such as boxing, Brazilian Jiu-Jitsu, Muay Thai, and wrestling. It requires strength, skill, and strategy, making it a thrilling and exciting sport to watch and practice. What do you enjoy most about MMA?'